# Lexical causatives in Lev 17-26

In a previous notebook the morphological causatives in Lev 17-26 (primarily *Hiphil* but also, to some extent, *Piel*) were analyzed. This notebook will focus on so-called lexical causatives.

In [1]:
#Dataset path
PATH = 'datasets/'

import pandas as pd
import numpy as np
import collections

from tf.app import use

In [2]:
A = use('bhsa', hoist=globals())

TF app is up-to-date.
Using annotation/app-bhsa commit 5fdf1778d51d938bfe80b37b415e36618e50190c (=latest)
  in C:\Users\Ejer/text-fabric-data/__apps__/bhsa.
Using etcbc/bhsa/tf - c r1.4 in C:\Users\Ejer/text-fabric-data
Using etcbc/phono/tf - c r1.1 in C:\Users\Ejer/text-fabric-data
Using etcbc/parallels/tf - c r1.1 in C:\Users\Ejer/text-fabric-data


#### Initial functions

In [3]:
def getLex(string, trans=True):
    
    if trans:
        string = string[:string.index('_')]
    else:
        string = string[string.index('_')+1:]
    return string

## Extracting verbs for annotation

All verbs function as verbs (phrase dependent part of speech = verb) are extracted from Leviticus 17-26. The morphological causatives already annotated are subtracted from this pool of verbs. Finally, all intransitive are subtracted, because, by definition, they cannot be causative.

In [7]:
all_verbs = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
   phrase function=Pred|PreO|PreS|PreC|PtcO
     word pdp=verb
'''

all_verbs = A.search(all_verbs)

  1.26s 936 results


In [82]:
unique_verbs = set([F.lex.v(r[3]) for r in all_verbs])
print(f'Count of all verbs: {len(unique_verbs)}')

Count of all verbs: 181


#### Subtracting morphological causatives

In [83]:
morph_caus = pd.read_csv(f'{PATH}Lev17-26_morphological_causitive.csv')
lexemes = list(morph_caus.iloc[:,0])
lexemes = [getLex(lex) for lex in lexemes]

In [84]:
verb_list = []

for verb in unique_verbs:
    if verb not in lexemes:
        verb_list.append(verb)
print(f'Count of all verbs minus morphological causatives: {len(verb_list)}')

Count of all verbs minus morphological causatives: 145


#### Subtracting intransitives

In order to subtract intransitives, we extract all relevant verbs in the corpus with their transitivity patterns on the basis of predefined transitivity frames.

The corpus is Genesis-Kings:

In [85]:
corpus = [book for book in F.otype.s('book') if book < T.nodeFromSection(('Isaiah',))]
sets={'corpus':corpus} # make set for searching

The predefined transitivity frames are as follows:

In [86]:
#Intransitive frame without object. Subject may be implicit or explicit.
intransitive = '''
corpus
 clause
 /without/
    phrase function=Objc
 /-/
    phrase function=Pred|PreS|PreC
     word lex#HJH[ pdp=verb
'''

#Transitive frame with object suffix.
transitive1 = '''
corpus
 clause
   phrase function=PreO|PtcO
     word lex#HJH[ pdp=verb
'''

#Transitive frame with lexical object
transitive2 = '''
corpus
 clause
   phrase function=Pred|PreS|PreC
     word lex#HJH[ pdp=verb
   phrase function=Objc
'''

In [87]:
frames = {'(S)':intransitive, '(S)-O_sfx':transitive1, '(S)-O':transitive2}

verb_constructions = {}

n=0
for fr in frames:
    
    clause = ''
    lex = ''
    stem = ''
    
    results = A.search(frames[fr], sets=sets, silent=True)
    
    for r in results:
        
        clause = r[1]
        lex = F.lex.v(r[3])

        verb_constructions[clause] = [lex, fr]
            
    n+=1
    print(f'... frame {n} completed')

... frame 1 completed
... frame 2 completed
... frame 3 completed


In [88]:
data = pd.DataFrame(verb_constructions).T
data.columns = ['lex','frame']

In [89]:
data.head()

,lex,frame
427556,RXP[,(S)
427557,>MR[,(S)
427561,VWB[,(S)
427562,BDL[,(S)
427568,>MR[,(S)


Are all relevant cases extracted? We can check this by comparing with a single search:

In [90]:
verbs = '''
corpus
 clause
  phrase function=Pred|PreO|PreS|PreC|PtcO
   word lex#HJH[ pdp=verb
'''

results_all_verbs = A.search(verbs, sets=sets)

  2.25s 31295 results


In [91]:
print(f'Number of cases in transitivity dataset: {len(data)}')
print(f'Number of all verbs regardless of transitivity frames: {len(results_all_verbs)}')
print(f'\nDifference: {len(results_all_verbs)-len(data)}')

Number of cases in transitivity dataset: 31295
Number of all verbs regardless of transitivity frames: 31295

Difference: 0


#### Computing transitivity

We are now ready to compute the transitivity of each lexeme. We work with two criteria:

1. Intransitive: No instances of lexical or suffix object
2. Transitives: Instances of either lexical or suffix object

First, we cross-tabulate lexeme and transitivity frame:

In [92]:
lex_frames = pd.crosstab(index=data.lex, columns=data.frame)
lex_frames.head()

frame,(S),(S)-O,(S)-O_sfx
lex,,,
<BD[,31,107,48
<BH[,2,0,0
<BR=[,1,0,0
<BR[,198,103,7
<BV[,1,2,1


In [93]:
intransitive_verbs = lex_frames[(lex_frames['(S)-O'] == 0) & (lex_frames['(S)-O_sfx'] == 0)]
transitive_verbs = lex_frames[(lex_frames['(S)-O'] > 0) | (lex_frames['(S)-O_sfx'] > 0)]

We can take a look at the most common of each:

In [94]:
intransitive_verbs.sort_values(by='(S)', ascending=False)[:10]

frame,(S),(S)-O,(S)-O_sfx
lex,,,
XNH[,128,0,0
XWH[,99,0,0
XRH[,58,0,0
GWR[,43,0,0
CKM[,41,0,0
LJN[,37,0,0
BRX[,36,0,0
>BH[,35,0,0
JYB[,28,0,0


In [95]:
print(f'Number of intransitive verbs: {len(intransitive_verbs)}')

Number of intransitive verbs: 320


In [96]:
transitive_verbs.sort_values(by=['(S)',('(S)-O')], ascending=False)[:10]

frame,(S),(S)-O,(S)-O_sfx
lex,,,
>MR[,3190,11,1
BW>[,1118,166,64
HLK[,889,19,2
<FH[,710,745,25
DBR[,591,81,5
JY>[,465,115,36
MWT[,422,37,52
<LH[,411,92,30
JCB[,398,14,3


In [97]:
print(f'Number of transitive verbs: {len(transitive_verbs)}')

Number of transitive verbs: 655


Finally, we can subtract the intransitive verbs from our list of verbs in Leviticus 17-26:

In [98]:
verb_list_upd = []

#We add the dummy-verb HJH[ to the list of intransitive verbs:
intrans = list(intransitive_verbs.index)
intrans.append('HJH[')

for verb in verb_list:
    if verb not in intrans:
        verb_list_upd.append(verb)
        
print(f'Number of verbs that are potentially causative: {len(verb_list_upd)}')

Number of verbs that are potentially causative: 125


### Predicting causativity

We assume that causative verbs are transative (or ditransitive) verbs where the Agent and the Patient is maximally distinguished in terms of affectedness. In other words, the verbal event will include an Agent that is not affected by the event but only instigating it, and a Patient which is caused to undergo the event.

For that reason we include affectedness data from another Notebook:

In [99]:
#A dictionary of columns to be imported as integers.
int_cols = {col:'Int64' for col in ['clause','Act_phr','Und1_phr','Und2_phr']}

aff_df = pd.read_csv(f'{PATH}Lev17-26.Volition_Affectedness_all_cor_3.csv', dtype=int_cols)
aff_df.head()

,Unnamed: 0,Unnamed: 0.1,clause,lex,Act_phr,Act_vol,Act_aff,Und1_phr,Und1_vol,Und1_aff,Und2_phr,Und2_vol,Und2_aff,comment
0,0,0,439650,DBR[,688348,y,n,688349,y,y,NaN,NaN,NaN,NaN
1,1,1,439651,>MR[,688350,y,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,439652,DBR[,688351,y,n,688352,y,y,NaN,NaN,NaN,NaN
3,3,3,439653,>MR[,688354,y,n,688355,y,y,NaN,NaN,NaN,NaN
4,4,4,439655,YWH[,688360,y,n,688358,n,n,NaN,NaN,NaN,NaN


We run through our list of verbs in Lev 17-26 and extract the affectedness annotation for all the verbs. The extracting is constrained by transitivity (that is, the verb must occur with a predicate/subject and an object) and affectedness (the verb must be annotated as either affected or not affected.

In [102]:
aff_dict = collections.defaultdict()

for lex in verb_list_upd:
    
    #We create a subset of the dataset for each of the lexemes in question:
    subset = aff_df[aff_df.lex == lex]

    for r in subset.iterrows():
        Act_aff = ''
        Und_aff = ''
        cl = r[1]['clause']

        #Check if Actor is subject/predicate and annotated for affectedness
        if F.function.v(r[1]['Act_phr']) in ['Pred','Subj','PreS','PreO','PtcO','PreC','Rela'] and r[1]['Act_aff'] in ['y','n']:
            Act_aff = r[1]['Act_aff']

            #Check if Undergoer1 is object and annotated for affectedness:
            objc = ['Objc','PreO','PtcO','Rela']
            if F.function.v(r[1]['Und1_phr']) in objc and r[1]['Und1_aff'] in ['y','n']:
                Und_aff = r[1]['Und1_aff']

            elif F.function.v(r[1]['Und2_phr']) in objc and r[1]['Und2_aff'] in ['y','n']:
                Und_aff = r[1]['Und2_aff']

        if Act_aff and Und_aff:
            aff_dict[cl] = [lex, Act_aff, Und_aff]

In [103]:
print(f'Transitive constructions found: {len(aff_dict)}')

Transitive constructions found: 315


Now we can check whether the lexemes can possibly be regarded as causatives. We do this by marking all constructions, where the predicate is not affected and the object is affected, with "True". All remaining constructions are marked with "False":

In [104]:
caus_dict = collections.defaultdict()

for cl in aff_dict:
    Act = aff_dict[cl][1]
    Und = aff_dict[cl][2]
    caus = False
    
    if Act == 'n' and Und == 'y':
        caus = True
        
    caus_dict[cl] = [aff_dict[cl][0], Act, Und, caus]

In [105]:
#caus_dict

Finally, we rearrange the dictionary in order to combine each lexeme with all its constructions marked as either causative ("True") and not causative ("False"):

In [106]:
lex_caus_dict = collections.defaultdict(list)

for cl in caus_dict:
    lex = caus_dict[cl][0]
    caus = caus_dict[cl][3]
    lex_caus_dict[lex].append(caus)
    
#lex_caus_dict

If a lexeme never occurs in a possible causative construction we regard as non-causative. Even if it occurs just one in a possible causative construction we regard it as causative:

In [107]:
caus_verbs = []
noncaus_verbs = []

for lex in lex_caus_dict:
    if True in lex_caus_dict[lex]:
        caus_verbs.append(lex)
    else:
        noncaus_verbs.append(lex)

In [108]:
print(f'Number of possible causative verbs: {len(caus_verbs)}')
print(f'Number of non-causative verbs: {len(noncaus_verbs)}')

Number of possible causative verbs: 56
Number of non-causative verbs: 34


In [109]:
noncaus_verbs.index('R>H[')

31

### Review

We are going to review all verbs to verify if the prediction based on affectedness holds. We begin with possible causative verbs.

We assign a dictionary for changes to be accounted:

In [126]:
changes = {}

#### Possible causative

In [34]:
query = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
   clause
     phrase function=Pred|PreS|PreC|PreO|PtcO
       word pdp=verb lex={}
'''

def show(lex, affect=caus_dict):
    results = A.search(query.format(lex), silent=True)
    clauses = [r[2] for r in results]
    aff = {}
    
    for cl in clauses:
        if cl in affect:
            if affect[cl][3] == True:
                aff[cl] = "lightgreen"
            else:
                aff[cl] = "red"
            
    A.show(results, highlights=aff)

The possible lexical causatives have been sorted into broad semantic categories: 

In [118]:
noncaus_verbs.index('NV<[')

24

##### Caused activity

The causing of an activity is a very common phenomenon in the corpus. Here, an object is caused to do something. The causative itself can, of course, be one of direct causation, permission and neglection. But these subtle differences are not accounted for here.

In [118]:
#show(caus_verbs[4])

In [ ]:
#show(caus_verbs[12])

In [125]:
#show(caus_verbs[15])

Literally, CPK "pour" seems to be a causative activity in the sense of "causing something to flow". On the other hand, in the uses found in Leviticus, the meaning seems to be somewhat different: "He/she has poured blood" or "he/she has shed blood". The implied meeming seems to be one of "causing someone to be extinct", hence, still causative.

In [127]:
#show(caus_verbs[16])

In [339]:
#show(caus_verbs[46])

RDP "to pursue" can be construed as "causing someone to flee"

In [121]:
#show(caus_verbs[13])

##### Causative of location

Some verbs desribe the causation of an object to be at a certain place:

The verb <ZB can be paraphrased as "cause them to remain":

In [181]:
#show(caus_verbs[44])

The verb BDL "separate" means to cause two objects to be geographically (or cognitively) separate:

In [147]:
#show(caus_verbs[26])

##### Transfer verbs

A large number of the causative verbs are transfer verbs which can largely be conceptualized as "causing something to be at a certain place" or "causing someone to become having something". These include the following verbs: 

In [81]:
show(caus_verbs[caus_verbs.index('NTN[')])

In [485]:
#show(caus_verbs[7])

In [113]:
#show(caus_verbs[11])

In [151]:
#show(caus_verbs[28])

In [168]:
#show(caus_verbs[37])

In [179]:
#show(caus_verbs[43])

Although LQX "take" is marked as non-causative in most of the cases because the Actor is affected and not only effecting the event, this phenomenon is only the result of the Actor both instigating the event and receiving the outcome. In many he cases he is "causing himself to have".

In [193]:
#show(caus_verbs[51])

QNH "buy" is somewhat similar to LQX "take" because the instigator may often be the recipient as well and he is therefore marked for affectedness. However, sometimes the focus seems to be on those bought from and therefore they are "caused to become not having" rather than the Actor himself having although this is implied.

In [189]:
#show(caus_verbs[49])

The verb NXL "take possession" seems to be a morphological causative. However, the only instance in Leviticus 17-26 is in Hitpael so it has not been included in the investigation of Piel and Hiphil verbs. Nevertheless, we retain this as a causative verb:

In [491]:
#show(caus_verbs[9])

##### Verbs of extinction

Verbs of extinction are very common as causatives. They can largely be paraphrased as "causing something/someone not to be". Verbs of this meaning are very common in the cultic texts of Leviticus. The classic example is of course HRG "kill":

In [72]:
show(caus_verbs[caus_verbs.index('HRG[')])

In [492]:
#show(caus_verbs[8])

In [493]:
#show(caus_verbs[10])

In [145]:
#show(caus_verbs[25])

In [161]:
#show(caus_verbs[33])

In [175]:
#show(caus_verbs[41])

In [186]:
#show(caus_verbs[47])

In [199]:
#show(caus_verbs[54])

In [201]:
#show(caus_verbs[55])

A curious case is GLX "shave" which implies the "extinction" of the beard.

In [500]:
#show(caus_verbs[48])

In [170]:
#show(caus_verbs[38])

And the concept of "shaving" is also applicaple to agriculture where the same meaning can be expressed as ZMR "prune" or "cut":

In [503]:
#show(caus_verbs[36])

Verbs such as "to break" may be regarded a subcategory of extinction. They are not completely removed or extinct but they have become broken:

In [74]:
#show(caus_verbs[caus_verbs.index('CBR[')])

In [505]:
#show(caus_verbs[3])

##### Factitives

Among the causatives are so-called factitives, that is, the causing of an entity to enter a new state, such as "make your souls detestable":

In [129]:
#show(caus_verbs[17])

In [135]:
#show(caus_verbs[20])

In [137]:
#show(caus_verbs[21])

In [80]:
show(caus_verbs[caus_verbs.index('XLL[')])

In [157]:
#show(caus_verbs[31])

In [159]:
#show(caus_verbs[32])

In [163]:
#show(caus_verbs[34])

In [173]:
#show(caus_verbs[40])

In [191]:
#show(caus_verbs[50])

The meaning of RJQ "empty" is somewhat unclear. It might be counted as a factitive:

In [139]:
#show(caus_verbs[22])

##### Speech verbs

Speech verbs are not normally considered causatives. However, YWH "order" seems to be causative, at least in some uses. In Lev 24:2 Moses is commanded "to order the Israelites so that they give you..." or better "to order the Israelites to give you". It is an indirect causation but nevertheless causative. The apparent exception in Lev 25:21 can be explained by the fact that "blessings" is an abstract concept so they cannot really be "affected". So the exceptions is not substantially challenging the interpretation of YWH as a causative.

In [149]:
#show(caus_verbs[27])

Another speech verb is JSR "instruct" or "admonish". It seems to be causative in the sense of "causing someone to learn":

In [177]:
#show(caus_verbs[42])

##### Other

The verb <CQ is predicted as causative. It means "to oppress" or "exploit" and may be causative. It depends on whether the object is really affected. Is the focus on the Actor and what he does or upon a changed situation of the Undergoer? If the situation of the Undergoer is worsened and he "becomes poor" or "weak" then the verb can be seen as a causative:

In [143]:
#show(caus_verbs[24])

A similar case is found with RDH "rule/tread" which is often associated with oppression. The same considerations apply to this verb because it may simply be an activity, but if the meaning is the changed situation of the Undergoer, it might be causative:

In [131]:
#show(caus_verbs[18])

In [165]:
#show(caus_verbs[35])

The verb >KL "eat" is predicted as causative which is surprising, as it is normally considered not-causative (e.g. Næss; Van Valin). However, in Leviticus there is one example where the verb is actually in a causative-like construction: "The land of your enemies shall eat/devour you" (Lev 26:38). Because "land" cannot be affected it functions like a regular agent. On the other hand, the object is clearly affected and, in fact, undergoing extinction. Therefore, while "eat" is not normally causative, the verb might be used in causative constructions.

In [59]:
show(caus_verbs[27])

In [127]:
changes['>KL['] = ''

The verb NKH "strike" is interesting. A standard interpretation of the verb is "to strike" and one would not expect the object to undergo a definite change of state, and therefore the verb is normally not regarded as causative. However, in Leviticus the normal meaning seems to be a fatal strike and therefore "causing someone not to be". However, in the larger corpus, this use may still be ununsual and we will therefore consider the standard meaning of the verb non-causative:

In [270]:
#show(caus_verbs[5])

In [128]:
changes['NKH['] = ''

The verb >PH "to bake" occurs once in a ditransitive construction, apparently having the meaning "bake something into bread", that is, it is more of a transformation than effecting bread. However, we may still regard "bake" as a creation verb:

In [272]:
#show(caus_verbs[23])

In [129]:
changes['>PH['] = ''

The verb JKX "reprove" is difficult but doet not seem to be causative. Perhaps the Undergoer is not really affected by the event and does not change place or status:

In [197]:
#show(caus_verbs[53])

In [134]:
changes['JKX['] = ''

The verb CPV "judge" is similar to JKX "reprove" and both may refer to a legal setting. Here, while the object is marked as affected it is not entirely clear how to rephrase the verb as a causative.

In [47]:
#show(caus_verbs[0])

In [130]:
changes['CPV['] = ''

The verb ZR< "sow" is predicted once to be a causative. However the phrase "to sow your seeds" may simply mean "to sow" so it is not enough evidence for regarding this verb as another than an activity.

In [49]:
#show(caus_verbs[6])

In [131]:
changes['ZR<['] = ''

KSH "to cover" seems to be causative, in the sense of "causing something (visible) to become hidden".

In [195]:
#show(caus_verbs[52])

See also <LM "hide"

In [123]:
#show(caus_verbs[14])

The verb <FH "to make" seems not to be causative in most cases because it is effecting an entity into existince rather than affecting an already existing entity. However, in a cultic context the verb is often used to express a sacrifice in the sense of "making something into a sacrifice". These cases may regarded as technical uses of the verb and may not influence the interpretation of the verb as non-causative:

In [53]:
#show(caus_verbs[10])

In [132]:
changes['<FH['] = ''

The verb <RL "leave unharvested" seems to be causative because the Undergoer is assumed to be undergoing an activity caused by the Actor. However, the verb is inherently negative and no action is ever carried out. The meaning of the causative may then be one of neglect because the Undergoer is intentionally neglected. In a sense the Undergoer is positively affected by the non-event because an event is cancelled.

In [153]:
#show(caus_verbs[29])

To sum up, so far we have made a few changes:

In [135]:
print(f'Changes made so far: {len(changes)}')

Changes made so far: 7


#### Not causatives

##### Cognitive verbs

Transitive cognitive verbs are verbs where the event is an internal, cognitive event. By consequence, the semantic object is not affected by the event and they are certainly not caused to undergo an activity or change state.

In [219]:
#show(noncaus_verbs[5])

In [227]:
#show(noncaus_verbs[9])

In [254]:
#show(noncaus_verbs[22])

The verb PRR "break" typically occurs with abstract concept, most often covenants or agreements. Therefore, the verb seems the annulment of a social relationship. The Undergoers will typically not be marked as affected because they are abstract concepts. Nevertheless, there seems to be a real-world effect of the event, so we should probably mark it as causative of "be broken".

In [231]:
#show(noncaus_verbs[11])

In [136]:
changes['PRR['] = 'caus'

##### Emotional verbs

This group of verbs is similar to the cognitive verbs in that the Undergoer is not affected by the internal event.

In [55]:
noncaus_verbs.index('JR>[')

25

In [56]:
show(noncaus_verbs[25])

In [213]:
#show(noncaus_verbs[2])

In [225]:
#show(noncaus_verbs[8])

In [545]:
#show(noncaus_verbs[31])

In [267]:
#show(noncaus_verbs[32])

##### Experiencing verbs

In [51]:
#show(noncaus_verbs[30])

##### Physical activities

Transitive physical activities are a diverse group of verbs where the Undergoer is not affected but perhaps the topic of the activity.

In [223]:
#show(noncaus_verbs[7])

In [229]:
#show(noncaus_verbs[10])

In [242]:
#show(noncaus_verbs[16])

In [244]:
#show(noncaus_verbs[26])

In [552]:
#show(noncaus_verbs[27])

In [553]:
#show(noncaus_verbs[29])

In [264]:
#show(noncaus_verbs[30])

MY> "find" is an event where the Undergoer is not affected by the event:

In [208]:
#show(noncaus_verbs[0])

##### Creation verbs

Creation verbs are characterized by the Undergoer not being affected but *effected*, that is, coming in to existence.

In [68]:
show(noncaus_verbs[23])

In [248]:
#show(noncaus_verbs[19])

In [245]:
#show(noncaus_verbs[17])

##### Affected agent

Some verbs occur freqently with an affected Agent. The Agent instigates an event but is himself the target of that event, such as "eat" or "wear".

In [221]:
#show(noncaus_verbs[6])

JRC "trample down": Seems to express possession. Therefore, the Agent is affected.

In [217]:
#show(noncaus_verbs[4])

To this group belongs verbs denoting sexual intercourse because both participants are affected:

In [560]:
#show(noncaus_verbs[18])

In [252]:
#show(noncaus_verbs[21])

##### Speech verbs

Normally the Undergoer of speech verbs is the topic which is obviously not affected by the speech.

In [215]:
#show(noncaus_verbs[3])

In [250]:
#show(noncaus_verbs[20])

In [ ]:
#show(noncaus_verbs[28])

##### Other

NFG "reach":

In [37]:
#show(noncaus_verbs[24])

In [38]:
show(noncaus_verbs[12])

The verb BYR "gather grapes" is a harvest verb and expresses a transfer of crops from the field to either a barn or the Actor himself. Probably, the Actor is conceived as receiving the crops thereby being affected by the transfer. In any case, we should probably understand the verb as causative:

In [239]:
#show(noncaus_verbs[14])

In [137]:
changes['BYR['] = 'caus'

The same is true of <LL "deal with",  QYR= "harvest", and >SP "gather":

In [567]:
#show(noncaus_verbs[13])
#show(noncaus_verbs[15])
#show(noncaus_verbs[23])

In [138]:
changes['<LL['] = 'caus'
changes['QYR=['] = 'caus'
changes['>SP['] = 'caus'

NV< "to plant" denotes a transfer of an entity into a place, therefore causative:

In [120]:
noncaus_verbs[noncaus_verbs.index('NV<[')]

'NV<['

In [139]:
changes['NV<['] = 'caus'

### Summing up

Changes made so far:

In [140]:
len(changes)

13

We create a dictionary with all reviewed verbs and their annotoation ("caus" or " "):

In [141]:
verb_dict = {}

for lex in caus_verbs:
    if lex in changes:
        verb_dict[lex] = changes[lex]
    else:
        verb_dict[lex] = 'caus'
        
for lex in noncaus_verbs:
    if lex in changes:
        verb_dict[lex] = changes[lex]
    else:
        verb_dict[lex] = ''

#verb_dict

Some verbs were filtered out initially because they were not transitive. Other verbs may have been filtered out because they were not annotated correctly in the affectedness dataset. At the end, we therefore need to review the remaining verbs:

In [142]:
remaining_verbs = []

for verb in verb_list_upd:
    if verb not in verb_dict:
        remaining_verbs.append(verb)
        
len(remaining_verbs)

35

### Review of remaining verbs

We do not expect the remaining verbs to be causative since they are either not transitive or have not been annotated for affectedness in the affectedness dataset. Nevertheless, we review the verbs one by one in order to check our hypothesis of the correspondence between transitivity and causation:

In [143]:
n=0

In [181]:
print(f'Case {n}')
show(remaining_verbs[n])
n+=1

Case 35


IndexError: list index out of range

GZL "tear off" seems to be a transfer of depriving an Undergoer from its possession, therefore causative:

In [146]:
changes['GZL['] = 'caus'

The same is true of GNB "steal":

In [174]:
changes['GNB['] = 'caus'

The verb PDH "buy off" seems to be inherently causative because it denotes a transfer. It has probably skipped the previous reviews because it only occurs as a Niphal in Lev 17-26 so, while it does have an affected Undergoer, the Undergoer functions as the syntactic subject and not the syntactic object as stipulated earlier. There is good reason to add this lexeme to our pool of causatives:

In [607]:
#show(remaining_verbs[20])

In [187]:
changes['PDH['] = 'caus'

The verb GRC "drive off" seems to be causative since the Actor causes the Undergoer to go away:

In [313]:
#show(remaining_verbs[18])

In [188]:
changes['GRC['] = 'caus'

The remaining verbs were considered non-causative. We can now add these verbs to our dictionary of reviewed verbs:

In [189]:
for lex in remaining_verbs:
    if lex in changes:
        verb_dict[lex] = changes[lex]
    else:
        verb_dict[lex] = ''

All relevant verbs have now been considered:

In [190]:
len(verb_dict)

125

## Export

The lexical causatives are exported according to their stem. If any given stem is not occuring in Leviticus 17-26 it has not been validated and will therefore not be annotated.

First, we search Lev 17-26 for all the relevant verb lexemes and retrieve their stems:

In [191]:
verbs = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
   phrase function=Pred|PreO|PreS|PreC|PtcO
     word pdp=verb
'''

verbs = A.search(verbs.format())

  1.03s 936 results


Lexeme and stem are concatonated and added to a dictionary with the annotation from the verb_dict: 

In [192]:
lex_stem = collections.defaultdict()

for r in verbs:
    lex = F.lex.v(r[3])
    stem = F.vs.v(r[3])
    
    if lex in verb_dict: #Check if lexeme occurs in verb_dict = lexical causative
        caus = verb_dict[lex]
        
    elif lex in intrans: #Check if lexeme occurs in intrans = intransitive = not causative
        caus = ''
        
    elif lex in lexemes: #Check if lexeme occurs in lexemes = morphogical causative - already annotated
        continue
        
    else:
        print(f'Missing verb: {lex}, {stem}')
        
    lex_stem[f'{lex}_{stem}'] = caus
    
lex_stem = pd.DataFrame([lex_stem]).T
lex_stem.head()

,0
<BD[_qal,
<CQ[_qal,caus
<FH[_nif,
<FH[_qal,
<LL[_piel,caus


In [193]:
lex_stem.to_csv(f'{PATH}Lev17-26_lexical_causative.csv')